In [1]:
import os
import pandas as pd

In [2]:
# 데이터 파일 불러오기
df=pd.read_csv(r"./data/감성분석/review_neg.csv",header=0,encoding="utf8",error_bad_lines=False)
df.head()

,doc,label
0,Overall we found the experience disappointing....,1
1,Jetstar insist frequently enough during the on...,1
2,Flight from JFK to Kiev was cancelled on July ...,1
3,The service is shocking. All was well until 1 ...,1
4,"Flew to Lisbon from Toronto July 27 2015, plan...",1


In [3]:
import re
import numpy as np
import json
import nltk
import itertools
import collections
import lda
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict

In [4]:
# 리뷰 데이터만 따로 지정
data_list=df['doc']
data_list

0        Overall we found the experience disappointing....
1        Jetstar insist frequently enough during the on...
2        Flight from JFK to Kiev was cancelled on July ...
3        The service is shocking. All was well until 1 ...
4        Flew to Lisbon from Toronto July 27 2015, plan...
                               ...                        
15352    MEL-BWN-BKI return. I have flown with this air...
15353    LHR to Nairobi. Punctual departure and arrival...
15354    PVG-XIY-PVG. Good fare on the outbound leg had...
15355    PVG-CKG. Booked flight at the last minute so w...
15356    London City-New York JFK via Shannon on A318 b...
Name: doc, Length: 15357, dtype: object

word_tokenize를 통해 단어 토큰화를 수행

In [5]:
# 영어 숫자 특수기호 남기기
review_list = []
for i in data_list:
    i = str(i)
    i = i.lower()
    text = re.sub('[^a-zA-Z0-9]',' ',i).strip()
    text = re.sub('[,]','',text)
    if(text != ''):
        if(text[0] !='?'):
            review_list.append(text)

In [6]:
review_list

['overall we found the experience disappointing  the seats were average  the wifi was fine  10mb for free  the entertainment system had a good variety of programs  the food was ordinary and the service was indifferent  on our return flight one of the stewardess s was incredibly rude  although the ground staff in dubai were fantastic  this is the 5th time we have flown this route and have had far better experiences with other airlines  singapore  malaysian  jal and saudi',
 'jetstar insist frequently enough during the online ticket purchase process that passengers must get to the airport 2 hours prior to departure which we dutifully did  arriving at melbourne airport a bit before 4 am  to catch the 6 05 am jetstar flight to sydney  when we arrived check in had just opened and there were only 2 staff checking baggage  got our baggage tag and got in the queue  then they proceeded to call out every single passenger whose flight were departing before ours  regardless of where they were in t

In [7]:
# review_list_df=pd.DataFrame(review_list)
# review_list_df.to_csv("./data/review_list.csv", encoding="utf-8")

불용어를 제거

In [8]:
airport_names=pd.read_csv(r"./data/stopwords_airport.csv",header=0,encoding="utf8",error_bad_lines=False)
airport_names.head()

,airport
0,auckland
1,christchurch
2,london
3,guangzhou
4,singapore


공항이름 제거

In [9]:
stop2=list(airport_names['airport'])

In [10]:
stop2

['auckland',
 'christchurch',
 'london',
 'guangzhou',
 'singapore',
 'paris',
 'dubai',
 'melbourne',
 'denpasar',
 'sydney',
 'brisbane',
 'myrtle beach',
 'laguardia',
 'jfk',
 'tel aviv',
 'kiev',
 'london gatwick',
 'florence',
 'athens',
 'larnaca',
 'toron',
 'lisbon',
 'lax',
 'hefei',
 'beijing',
 'cincinnati',
 'phoenix',
 'manchester',
 'jfk new york',
 'vie',
 'los angeles',
 'milan',
 'lhr',
 'vienna',
 'geneva',
 'fort lauderdale',
 'fra',
 'clermont-ferrand',
 'newark',
 'hong kong',
 'new york',
 'auh',
 'raleigh-durham',
 'denver',
 'berlin',
 'dar es salaam',
 'ams',
 'jakarta',
 'bengkulu',
 'zagreb',
 'warsaw',
 'rome',
 'sao paulo',
 'frankfurt',
 'dxb',
 'mct',
 'kigali',
 'doha',
 'bangkok',
 'amsterdam',
 'amman',
 'johannesburg',
 'hog',
 'yul',
 'phuket',
 'london stansted',
 'paphos',
 'ankara',
 'istanbul',
 'm찼laga',
 'tokyo',
 'bos',
 'idaho falls',
 'hanoi',
 'atlanta',
 'montreal',
 'bangalore',
 'delhi',
 'clearwater st pete fl',
 'syracuse ny',
 'st. t

In [12]:
stop_words = 'flight','airline','class','time','service','hour','plane','return','very','however','time','bit','as','my','then','never','next','no','due','one','us','ife','entertainment','airplane','arrive','back','airport','business','economy','even','use','air','friend','experience','new','offer','aircraft','way','trip','go','bag','ticket','home','film','june','warm','bkk','jfk','a320','a319','cx','kl','icn','july','not','only','so','first','more','other','flight.','just','again','much','also','really','well','better'
nnp_list = []

for i in review_list:
    origin_words = nltk.word_tokenize(i)
    data_pos = nltk.pos_tag(origin_words)
    words_nnp = [word for word,pos in data_pos if pos in ['NN','NNP','VBG','JJ','JJS','JJR']]
    words_nnp = [w for w in words_nnp if not w in stopwords.words('english')]
    words_nnp = [w for w in words_nnp if not w in stop_words]
    words_nnp = [w for w in words_nnp if not w in stop2]
    nnp_list.append(words_nnp)

In [13]:
print(nnp_list[0:5])

[['overall', 'disappointing', 'average', 'wifi', 'fine', 'free', 'system', 'good', 'variety', 'food', 'ordinary', 'indifferent', 'stewardess', 'rude', 'ground', 'staff', 'fantastic', '5th', 'route', 'malaysian', 'jal', 'saudi'], ['jetstar', 'insist', 'online', 'purchase', 'process', 'arriving', 'jetstar', 'check', 'staff', 'checking', 'baggage', 'baggage', 'tag', 'queue', 'single', 'passenger', 'departing', 'queue', 'point', 'front', 'line', 'jet', 'star', 'pulling', 'getting', 'riot', 'check', 'guy', 'less', 'security', 'care', 'pay', 'extra', 'stuff', 'food', 'seat', 'nuisance', 'fine', 'pay', 'lack', 'coordinated', 'path', 'baggage', 'poor', 'organization', 'acceptable', 'cheap', 'airfare', 'worth', 'anxiety', 'maddening', 'angry', 'resentful', 'qantas', 'offshoot', 'end', 'bother', 'early', 'part', 'difference', 'trust', 'least', 'domestic', 'departing', 'get', 'flag', 'staff', 'member', 'boarding', 'right', 'front', 'queue', 'bye', 'bye', 'poor', 'basic', 'priority', 'stressful', 

In [14]:
nnp_list_1d = list(itertools.chain.from_iterable(nnp_list))
nnp_list_1d[0:20]

['overall',
 'disappointing',
 'average',
 'wifi',
 'fine',
 'free',
 'system',
 'good',
 'variety',
 'food',
 'ordinary',
 'indifferent',
 'stewardess',
 'rude',
 'ground',
 'staff',
 'fantastic',
 '5th',
 'route',
 'malaysian']

표제어 추출

In [15]:
tokenized_doc=[]
for i in nnp_list_1d:
    a=WordNetLemmatizer().lemmatize(i)
    tokenized_doc.append(a)

In [16]:
tokenized_doc[0:20]

['overall',
 'disappointing',
 'average',
 'wifi',
 'fine',
 'free',
 'system',
 'good',
 'variety',
 'food',
 'ordinary',
 'indifferent',
 'stewardess',
 'rude',
 'ground',
 'staff',
 'fantastic',
 '5th',
 'route',
 'malaysian']

In [17]:
# # 역토큰화 (토큰화 작업을 되돌림)
# detokenized_doc = []
# for i in range(len(text)):
#     t = ' '.join(tokenized_doc[i])
#     detokenized_doc.append(t)

# text['content'] = detokenized_doc # 다시 text['headline_text']에 재저장

In [18]:
# reviews[:5]

In [19]:
# reviews1=pd.DataFrame(reviews, columns=['content'])
# reviews1

사용빈도가 1 이하인 단어 삭제

In [20]:
# texts = [[word for word in document.lower().split() if word not in stop] for document in reviews]

In [21]:
# test=texts

In [22]:
# test

In [23]:
# test=sum(test,[])

In [24]:
# test

In [25]:
# len(test)

In [26]:
testset=[t.split() for t in tokenized_doc]

In [27]:
testset

[['overall'],
 ['disappointing'],
 ['average'],
 ['wifi'],
 ['fine'],
 ['free'],
 ['system'],
 ['good'],
 ['variety'],
 ['food'],
 ['ordinary'],
 ['indifferent'],
 ['stewardess'],
 ['rude'],
 ['ground'],
 ['staff'],
 ['fantastic'],
 ['5th'],
 ['route'],
 ['malaysian'],
 ['jal'],
 ['saudi'],
 ['jetstar'],
 ['insist'],
 ['online'],
 ['purchase'],
 ['process'],
 ['arriving'],
 ['jetstar'],
 ['check'],
 ['staff'],
 ['checking'],
 ['baggage'],
 ['baggage'],
 ['tag'],
 ['queue'],
 ['single'],
 ['passenger'],
 ['departing'],
 ['queue'],
 ['point'],
 ['front'],
 ['line'],
 ['jet'],
 ['star'],
 ['pulling'],
 ['getting'],
 ['riot'],
 ['check'],
 ['guy'],
 ['le'],
 ['security'],
 ['care'],
 ['pay'],
 ['extra'],
 ['stuff'],
 ['food'],
 ['seat'],
 ['nuisance'],
 ['fine'],
 ['pay'],
 ['lack'],
 ['coordinated'],
 ['path'],
 ['baggage'],
 ['poor'],
 ['organization'],
 ['acceptable'],
 ['cheap'],
 ['airfare'],
 ['worth'],
 ['anxiety'],
 ['maddening'],
 ['angry'],
 ['resentful'],
 ['qantas'],
 ['offshoo

In [28]:
from gensim import corpora

dictionary = corpora.Dictionary(testset)

In [29]:
print(dictionary)

Dictionary(14655 unique tokens: ['overall', 'disappointing', 'average', 'wifi', 'fine']...)


In [30]:
corpus = [dictionary.doc2bow(text) for text in testset]

In [31]:
# # LDA 토픽모델링 수행
# ## 토픽 개수 다양하게 돌리면서 해보기
# from gensim.models import LdaModel

# ldamodel = LdaModel(corpus, num_topics = 7, id2word=dictionary, passes=30)

In [32]:
# topics = ldamodel.show_topics()

# for topic in topics:
#     print(topic)

In [33]:
# LDA 토픽모델링 수행
## 토픽 개수 다양하게 돌리면서 해보기
from gensim.models import LdaModel

ldamodel8 = LdaModel(corpus, num_topics = 8, id2word=dictionary, passes=30, iterations=400)

1 ~ 10 까지 반복

In [34]:
word_dict = {}

for i in range(8):
    words = ldamodel8.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,good,check,staff,boarding,seat,crew,food,cabin
1,poor,gate,meal,luggage,inflight,extra,old,leg
2,bad,front,delay,long,day,available,last,room
3,drink,water,flying,little,nothing,board,departure,uncomfortable
4,legroom,baggage,system,ok,worst,going,rude,late
5,connection,comfortable,small,hotel,many,cost,hr,customer
6,breakfast,friendly,night,lounge,choice,tv,problem,full
7,money,great,lot,waiting,terrible,work,awful,connecting
8,free,attendant,able,average,wife,hot,flew,quality
9,different,explanation,desk,arrival,seating,international,information,ground


In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수8).csv', encoding="utf-8")

In [ ]:
word_dict = {}

for i in range(2):
    words = ldamodel2.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수2).csv', encoding="utf-8")

In [ ]:
word_dict = {}

for i in range(3):
    words = ldamodel3.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수3).csv', encoding="utf-8")

In [ ]:
word_dict = {}

for i in range(4):
    words = ldamodel4.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수4).csv', encoding="utf-8")

In [ ]:
word_dict = {}

for i in range(5):
    words = ldamodel5.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수5).csv', encoding="utf-8")

In [ ]:
word_dict = {}

for i in range(6):
    words = ldamodel6.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수6).csv', encoding="utf-8")

In [ ]:
word_dict = {}

for i in range(7):
    words = ldamodel7.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수7).csv', encoding="utf-8")

In [ ]:
word_dict = {}

for i in range(8):
    words = ldamodel8.show_topic(i, topn = 51)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
    
pd.DataFrame(word_dict)

In [ ]:
word_per_topic=pd.DataFrame(word_dict)
word_per_topic.to_csv('./data/LDA토픽별 단어/토픽별 단어 파일_부정/LDA단어(토픽개수8).csv', encoding="utf-8")

In [105]:
# words_list=tokenized_doc

In [106]:
# # 토픽 모델링 과정.
# # WordCount
# word_count = collections.Counter(words_list)
# result = word_count.most_common(1000)
# print(len(word_count))
# print(result)

In [107]:
# # Countervoctorzier & LDA
# c_vetorizer = CountVectorizer(analyzer='word')
# count = c_vetorizer.fit_transform(words_list)

# model = lda.LDA(n_topics = 8, n_iter = 2000, random_state = 1)
# model.fit(count)

In [108]:
# # LDA result
# topic_vocab = c_vetorizer.get_feature_names()
# topic_word = model.topic_word_
# n_top_word = 100
# dist = []
# for i, topic_dist in enumerate(topic_word):
#     dist.append(topic_dist)
#     topic_words = np.array(topic_vocab)[np.argsort(topic_dist)][:-n_top_word:-1]
#     print('Topic', i+1, topic_words)

In [ ]:
import pyLDAvis

pyLDAvis.__version__

In [ ]:
import pyLDAvis.gensim as gensimvis

In [ ]:
prepared_data = gensimvis.prepare(ldamodel2, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_2).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel3, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_3).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel4, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_4).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel5, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_5).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel6, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_6).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel7, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_7).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel8, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_8).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel9, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_9).html')

In [ ]:
prepared_data = gensimvis.prepare(ldamodel10, corpus, dictionary)

pyLDAvis.save_html(prepared_data, 'LDA_Visualization(Topic_num_10).html')

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

coherencesT=[]
perplexitiesT=[]

coherencesT.append(coherence)

perplexitiesT.append(ldamodel1.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel2, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel2.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel3, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel3.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel4, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel4.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel5, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel5.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel6, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel6.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel7, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel7.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel8, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel8.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel9, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel9.log_perplexity(corpus))

In [ ]:
cm = CoherenceModel(model=ldamodel10, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()

coherencesT.append(coherence)

perplexitiesT.append(ldamodel10.log_perplexity(corpus))

In [ ]:
measure.T.to_csv('./data/Coherence, Perplexity/Measuring_부정.csv', encoding="utf-8")